In [1]:
from elasticsearch import Elasticsearch
import json
from sentence_transformers import SentenceTransformer

## Step 1: Prepare the documents

In [2]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

## Step 2: Create Embeddings using Pretrained Models

In [3]:
model = SentenceTransformer("all-mpnet-base-v2")


In [4]:
embeddings = model.encode([doc['text'] for doc in documents])

In [5]:
operations = []
for doc, emb in zip(documents, embeddings):
    doc['text_vector'] = emb.tolist()
    operations.append(doc)

## Step 3: Setup ElasticSearch Connection

In [6]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '634ffde4922a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'EfnPN_SvQ7yXaAkwcMYNUA', 'version': {'number': '8.17.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'a091390de485bd4b127884f7e565c0cad59b10d2', 'build_date': '2025-02-28T10:07:26.089129809Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Step 4: Create Mappings and Index

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "section": {
                "type": "text"
            },
            "question": {
                "type": "text"
            },
            "course": {
                "type": "keyword"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
       }
    }
}

index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Step 5: Add documents to the index

In [8]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)
    

## Step 6: Create end user query

In [9]:
search_term = "Windows or mac?"
vector_search_term = model.encode(search_term)

## Step 7: Perform Semantic Search

In [10]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [11]:
response = es_client.search(index=index_name, knn=query, source=["course", "section", "question", "text"])
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'hJdxoJUBrMiDmuOl3iSx',
  '_score': 0.7149744,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'l5dxoJUBrMiDmuOl7SeT',
  '_score': 0.613544,
  '_source': {'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at one shop:\nYou can use anaconda which has all built in services like pycharm, jupyter\nAdded by Khaja Zaff

## Step 8: Advanced Search

In [29]:
query_complex = {
        'query_vector': vector_search_term,
        'field': 'text_vector',
        'k': 5,
        'num_candidates': 10000,
        'filter': {
            "term": {
                "course": "data-engineering-zoomcamp"
            }
        }
    }


response = es_client.search(index=index_name, explain=True, body={"knn": query_complex})
response['hits']['hits']


/tmp/ipykernel_2181/2483766083.py:14: DeprecationWarning: Received 'explain' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es_client.search(index=index_name, explain=True, body={"knn": query_complex})


[{'_shard': '[course-questions][0]',
  '_node': 'EkxI_8xUQr6014cVmJkz6A',
  '_index': 'course-questions',
  '_id': 'hJdxoJUBrMiDmuOl3iSx',
  '_score': 0.7149744,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
